# 📰 AI News Fetching & Storage Script

## 🔍 Overview:
This script fetches AI-related news articles from an API, processes them into a structured format, and stores them in a **SQLite database**. If database storage fails, the data is saved as a **CSV backup**. The script can also be run **periodically**.

---

## 📌 Script Breakdown:

### **1️⃣ Fetch News from API**
- **Loads API key** securely from a `.env` file.
- **Sends a request** to the API to fetch **100 AI-related news articles**.
- **Handles API errors** (connection issues, HTTP errors, or invalid API responses).

### **2️⃣ Process API Response**
- Converts **API JSON response** into a **pandas DataFrame**.
- Ensures data is structured with relevant columns.

### **3️⃣ Store Data in SQLite Database**
- **Creates a table** if it doesn't exist.
- **Inserts new news articles** into the database.
- **Uses transactions** to ensure data consistency.
- If database write **fails**, **backs up data as CSV**.

### **4️⃣ Run Periodically (Optional)**
- Can **run once** or **repeat automatically** every **X seconds**.
- Uses **cron (Linux/MacOS)** or **Task Scheduler (Windows)** for scheduling.

---

## 🎯 Key Takeaways:
✅ **Automates news collection** from an external source.  
✅ **Ensures secure API access** using `.env` variables.  
✅ **Stores data in a database**, with **CSV backup** for reliability.  
✅ **Can run on a schedule** for continuous updates.  



In [5]:
#!/usr/bin/env python3

# Necessary variables
DB_FILE="news.db"
BASE_URL="http://api.mediastack.com/v1"

# Imports
import pandas as pd  # To work with tabular data
import sqlite3  # To work with the SQLite database
import requests  # To make API request
import json  # To parse JSON response from API
import os
from dotenv import load_dotenv  # To read environment variables
from time import sleep  # To run periodically
from datetime import datetime

# Attributes of a news article as per API specs - all of them will be stored
columns = ['author', 'title', 'description', 'url', 'source', 'image', 'category', 'language', 'country', 'published_at']

# Function to return current time for basic logging
def curr_time():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Fetch news from API
def fetch():
    # Load environment variables from the .env file
    load_dotenv()
    # Read API token from environment
    API_KEY = os.getenv("API_KEY")
    
    # Ensure API key is loaded correctly
    if not API_KEY:
        print(f"[{curr_time()}] Error: API Key not found! Check your .env file.")
        raise SystemExit("Missing API Key")
    
    # Construct the API request URL
    num_articles = 100
    url = f"{BASE_URL}/news?access_key={API_KEY}&languages=en&keywords=ai&sort=published_desc&limit={num_articles}"
    
    print(f"[{curr_time()}] Collecting news...")
    
    try:
        response = requests.get(url)  # Sending actual HTTP request
        result = response.json()  # Parsing the response
    except requests.exceptions.RequestException as e:
        # Handle connection problems
        print(f"[{curr_time()}] API request failed: connection error")
        raise SystemExit(e)

    # Check if HTTP request is successful
    if response.status_code != 200:
        print(f"[{curr_time()}] API request failed: HTTP error")
        raise SystemExit(f"Error {response.status_code}: {response.reason}")

    # Check if the API server returned an error
    if "error" in result:
        print(f"[{curr_time()}] API request failed: {result['error']['message']}")
        raise SystemExit(result["error"])

    print(f"[{curr_time()}] News collected, processing...")
    return result

# Store the results in a tabular form in a pandas dataframe
def process(result):
    # Creating a dataframe from a list of dicts (see API specs)
    df = pd.DataFrame.from_records(result["data"], columns=columns)
    return df

def store(df):
    # Store the data in a SQLite database - a lightweight SQL database
    conn = sqlite3.connect(DB_FILE)

    # Create the "news" table if it doesn't exist
    query_columns = ", ".join([f"{c} TEXT" for c in columns])
    create_db_query = f"CREATE TABLE IF NOT EXISTS news ({query_columns})"
    conn.execute(create_db_query)

    # Writing the fetched news into the database
    print(f"[{curr_time()}] Saving into the database...")
    conn.execute('BEGIN')  # Begin a transaction
    try:
        df.to_sql('news', con=conn, if_exists="append", index=False)
        conn.commit()  # Commit transaction
        print(f"[{curr_time()}] News saved successfully")
    except Exception as e:
        print(e)
        print(f"[{curr_time()}] Failed to save the news to the database, saving to CSV...")
        conn.rollback()  # Undo transaction if there's an error
        df.to_csv(f"news_{datetime.now().strftime('%Y_%m_%d')}.csv", index=False)
        print(f"[{curr_time()}] News saved to CSV")
    finally:
        conn.close()  # Close the DB connection

def main():
    result = fetch()
    df = process(result)
    store(df)

def run_periodically(interval_seconds):
    while True:
        try:
            main()
        except SystemExit as e:
            print(f'[{curr_time()}] Error: "{e}". Repeating in {interval_seconds} seconds')
        finally:
            sleep(interval_seconds)

if __name__ == "__main__":
    main()
    # To run periodically, use:
    # run_periodically(86400)  # Runs every 24 hours


[2025-02-09 22:55:22] Collecting news...
[2025-02-09 22:55:24] News collected, processing...
[2025-02-09 22:55:24] Saving into the database...
[2025-02-09 22:55:25] News saved successfully


# 🗄️ Checking Tables in SQLite Database

## 🔍 Overview:
This script connects to an **SQLite database** and retrieves the list of available tables.

---

## 📌 Script Breakdown:

### **1️⃣ Define Database Path**
- Specifies the **full path** to the SQLite database (`news.db`).

### **2️⃣ Connect to Database**
- Establishes a **connection** to the database.
- Creates a **cursor** to execute SQL queries.

### **3️⃣ Fetch Table Names**
- Queries SQLite's **system metadata** (`sqlite_master`) to get a list of **all tables**.
- Prints the **available tables** in the database.

### **4️⃣ Close Connection**
- Ensures the **database connection is closed** after execution.

---

## 🎯 Key Takeaways:
✅ **Verifies database connectivity** before executing queries.  
✅ **Lists all tables** to confirm database structure.  
✅ **Prepares for further queries** (e.g., retrieving stored news articles).  


In [7]:
import sqlite3

# Define the database path
db_path = r"C:\Users\bryan\Desktop\RavenPack\news.db"

# Connect to SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Check available tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the tables found
print("Tables in the database:", tables)

# Close connection
conn.close()



Tables in the database: [('news',)]


# 📰 Fetching News Data from SQLite Database

## 🔍 Overview:
This script **connects to an SQLite database** and retrieves the **first 5 rows** from the `news` table.

---

## 📌 Script Breakdown:

### **1️⃣ Define Database Path**
- Specifies the **file path** to the SQLite database (`news.db`).

### **2️⃣ Connect to SQLite**
- Establishes a connection to the **SQLite database**.
- Allows executing **SQL queries** using `pandas`.

### **3️⃣ Retrieve Data**
- Runs an **SQL query** to select **the first 5 rows** from the `news` table.
- Stores the result in a **pandas DataFrame**.

### **4️⃣ Close Connection**
- Ensures the **database connection is closed** after retrieving data.

### **5️⃣ Display Results**
- Prints the retrieved **news data**.

---

## 🎯 Key Takeaways:
✅ **Confirms data exists** in the database.  
✅ **Uses SQL & pandas** for efficient data handling.  
✅ **Prepares for further analysis** or visualisation.  


In [4]:
import sqlite3
import pandas as pd

# Define the path to your database file
db_path = r"C:\Users\bryan\Desktop\RavenPack\news.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Query to select the first 5 rows from the 'news' table
query = "SELECT * FROM news LIMIT 5"
df = pd.read_sql(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)


             author                                              title  \
0     Romain Dillet  Investments in French AI ecosystem reach $85B ...   
1   Alexander Brown  The Scottish politician who cost taxpayer £100...   
2              None                          How to read burnt scrolls   
3  Kentigern Minggu  Sarawak’s Land and Survey to embrace digital t...   
4              None  Meta's AI Capex Can Surge +$65B In 2025 - That...   

                                         description  \
0  Canadian investment firm Brookfield plans to i...   
1  There were also accusations some of the questi...   
2  How AI can decipher the writing in a charred p...   
3  KUCHING (Feb 9): The Sarawak Land and Survey D...   
4  Meta's AI Capex Can Surge +$65B In 2025 - That...   

                                                 url         source  \
0  https://techcrunch.com/2025/02/09/investments-...     TechCrunch   
1  https://www.scotsman.com/news/politics/the-sco...       scotsman   
2  ht